<a href="https://colab.research.google.com/github/yasiruchamuditha/ML_Data_Assignment/blob/main/Youtube_Trending_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
#The URL of the data set stored GitHub Repositiory
url = "https://raw.githubusercontent.com/Rush99x/Machine-learning-model-train/main/USvideos.csv"

# Load the dataset into a Pandas DataFrame
df = pd.read_csv(url)

# Display the first few rows of the dataset
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [ ]:
#Data Pre Processing
#Remove duplicate values
df = df.drop_duplicates()

# Handle missing values in specific columns
columns_to_fillna = ['description', 'title', 'tags', 'views', 'likes', 'dislikes', 'comment_count']
df[columns_to_fillna] = df[columns_to_fillna].fillna('')


<ipython-input-45-1691a12a52da>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns_to_fillna] = df[columns_to_fillna].fillna('')


In [ ]:
# Feature engineering
# Drop columns that are not needed for training the model
df = df.drop(['video_id', 'trending_date', 'publish_time', 'thumbnail_link'], axis=1)

# Convert categorical variables to numerical format
le = LabelEncoder()
df['title'] = le.fit_transform(df['title'])
df['tags'] = le.fit_transform(df['tags'])

# Feature engineering - creating a new feature 'trending' (target variable)
# Assuming that if 'likes' are greater than a certain threshold, the video is trending
threshold_likes = 10000
df['trending'] = (df['likes'] > threshold_likes).astype(int)

In [ ]:
#Train Test Split
# Select features for training the model
features = ['title', 'tags', 'views', 'dislikes', 'comment_count']
X = df[features]
y = df['trending']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Model Selection
# Build a Random Forest Classifier model - Ensemble Learning /Highly Accurate /Reduced Overfitting /Feature Importance /Feature Importance
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Display classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3008
           1       0.97      0.98      0.97      5173

    accuracy                           0.96      8181
   macro avg       0.96      0.96      0.96      8181
weighted avg       0.96      0.96      0.96      8181



In [ ]:
#Deployement
# Create a new sample input for testing
new_input = {
    'channel_title': 'Netflix',
    'tags': 50,
    'views': 5260000078,
    'dislikes': 100000,
    'comment_count': 200007,  # Assuming this is the correct mapping
}


In [ ]:
# Create a DataFrame with the new input
new_input_df = pd.DataFrame([new_input])

In [ ]:
# Ensure all columns are present in the new input DataFrame
missing_columns = set(X_train.columns) - set(new_input_df.columns)
for column in missing_columns:
    new_input_df[column] = 0  # Add missing columns with value 0

In [ ]:
# Reorder columns to match the order during training
new_input_df = new_input_df[X_train.columns]

In [ ]:
# Make predictions
prediction = model.predict(new_input_df)

In [ ]:
# Display the prediction
if prediction[0] == 1:
    print("Trending")
else:
    print("Not Trending")

Trending
